In [1]:
import random
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
import nltk
import csv
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from matplotlib import pyplot as plt
import datetime
import time
import lightgbm
import spacy
nlp_fr = spacy.load('fr_core_news_md', disable=["tagger", "parser", "ner", "textcat"])
nlp_en = spacy.load('en_core_web_md', disable=["tagger", "parser", "ner", "textcat"])
from tqdm import tqdm
import os
import networkx as nx
import pickle
from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin
import sklearn.feature_extraction.text as fe
from sklearn.decomposition import NMF, LatentDirichletAllocation

Using TensorFlow backend.


In [2]:
nltk.download('punkt') # for tokenization
nltk.download('stopwords')
stpwds_en = set(nltk.corpus.stopwords.words("english"))
stpwds_fr = set(nltk.corpus.stopwords.words("french"))
stemmer = nltk.stem.PorterStemmer()
spacy_stopwords_en = spacy.lang.en.stop_words.STOP_WORDS        
spacy_stopwords_fr = spacy.lang.fr.stop_words.STOP_WORDS        
spacy_stopwords_union = set(spacy_stopwords_en).union(set(spacy_stopwords_fr))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fvice\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fvice\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
corpus_path = r"pickles/corpus.PICKLE"
ids_path = r"pickles/IDs.PICKLE"
if os.path.exists(corpus_path):
    with open(corpus_path, 'rb') as f:
        corpus = pickle.load(f)
    f.close()
    with open(ids_path, 'rb') as f:
        ids = pickle.load(f)
    f.close()
else:
    directory = r"node_information/text/"
    corpus = []
    ids = []
    for filename in tqdm(os.listdir(directory)):
        with open(directory + filename, 'r', encoding='UTF-8', errors='ignore') as f:
            doc_string = []
            for line in f:
                [doc_string.append(token) for token in line.lower().strip().split(" ")]
            corpus.append(' '.join(doc_string))
            ids.append(filename[:-4])
    with open(corpus_path, '+wb') as f:
        pickle.dump(corpus, f)
    f.close()
    with open(ids_path, '+wb') as f:
        pickle.dump(ids, f)
    f.close() 

node_info = pd.DataFrame({'ID': ids, 'Corpus': corpus})
print("Training node info shape: {}".format(node_info.shape))

Training node info shape: (33226, 2)


In [131]:
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
def calculate_languages_ratios(text):
    languages_ratios = []
    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens]
    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        words_set = set(words)
        common_elements = words_set.intersection(stopwords_set)
        languages_ratios.append(len(common_elements))
    if sum(languages_ratios) == 0:
        return np.zeros(len(languages_ratios))
    return np.array(languages_ratios)/sum(languages_ratios)

In [134]:
languages_nltk = {}
loop_size = len(node_info['Corpus'])
for i in tqdm(range(loop_size), total=loop_size, leave=True, position=0):
    try:
        languages_nltk[node_info['ID'][i]] =  calculate_languages_ratios(node_info['Corpus'][i])
    except:
        languages_nltk.append(None)

  2%|█▍                                                                            | 600/33226 [00:18<12:21, 43.98it/s]

AttributeError: 'dict' object has no attribute 'append'